<a href="https://colab.research.google.com/github/ntu-dl-bootcamp/deep-learning-2026/blob/main/SESSION2/session2_part3.ipynb" target="_blank"><img alt="Open In Colab" src="https://colab.research.google.com/assets/colab-badge.svg"/></a>

# Challenge - House Price Prediction Using Neural Network

Using the concepts learnt in the session so far, try to solve a classic regression problem - predicting the price of a house based on its features!

You can take the help of parts 1 and 2 of the notebook wherever necessary.

### Imports

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from torch import nn

### Set device

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

## Challenge 1: Dataset and DataLoader

### Loading Data

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing_data = fetch_california_housing()
# print(housing_data)
print("Number of Features: ", len(housing_data.data[0]))
print("Number of Data Samples: ", len(housing_data.data))

### Complete Custom Dataset Class

In [ ]:
# Refer: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

class CustomDataset(Dataset):
    def __init__(self, data_source):
        self.data_source =data_source
        self.features = torch.tensor(self.data_source.data, dtype=torch.float32)
        self.labels = torch.tensor(self.data_source.target, dtype=torch.float32)


    def __len__(self):
        return len(self.data_source.data)

    def __getitem__(self, idx):
        feature, label = self.features[idx], self.labels[idx]  # Use idx (index) to reference each input row in the dataset
        return feature, label


### Specify the batch_size

In [ ]:
batch_size = 50 # try different values like 1, 2, 10, 100

### Declare Custom Dataset Class Object

In [ ]:
custom_dataset = CustomDataset(housing_data)

### Split the dataset into train and test

In [ ]:
from torch.utils.data import random_split

split_ratio = 0.9 # 9:1 ratio for train:test
train_size = int(split_ratio * len(custom_dataset))
test_size = len(custom_dataset) - train_size

#Refer to torch.utils.data.random_split() in https://pytorch.org/docs/stable/data.html

housing_train_dataset, housing_test_dataset = random_split(custom_dataset, [train_size, test_size])

print(len(housing_train_dataset))
print(len(housing_test_dataset))


### Intialize the data loader

In [ ]:
train_housing_data_loader = DataLoader(housing_train_dataset, batch_size=batch_size, shuffle=True)
test_housing_data_loader = DataLoader(housing_test_dataset, batch_size=batch_size, shuffle=True)

### Execute the below code to check Dataloader  - Try changing the batch_size!

In [ ]:
for  id, batch_data in enumerate(train_housing_data_loader):

    batch_features, batch_labels = batch_data[0], batch_data[1]
    print("batch_features: ",  batch_features)
    print("batch_labels: ", batch_labels)

    break

## Challenge 2: Building Neural Network

### Complete Code for NeuralNetwork class below

In [ ]:
class HousingNueralNetwork(nn.Module): # Inherit from nn.Module
    def __init__(self):
        super().__init__()


        # Define a sequential container of layers in network
        #You are free to choose number of layers and number of neurons in each layer
        self.layer_stack = nn.Sequential(
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 2),
            nn.ReLU(),
            nn.Linear(2, 1),
        )

    def forward(self, x):

        output = self.layer_stack(x)
        return output

#Create Object of HousingNueralNetwork class
housing_model = HousingNueralNetwork()

#Send model to DEVICE
housing_model = housing_model.to(DEVICE)

#print model
print(housing_model)

## Challenge 3 - Network Training

### Specify Hyperparameter - learning rate and number of epochs

In [ ]:
learning_rate = 1e-3 # Try different values like 0.01 , 0.002. etc
epochs = 5

### Specify the loss function

In [ ]:
housing_loss_fn = nn.MSELoss()

### Specify Optimizer

In [ ]:
from torch import optim

housing_optimizer =  optim.SGD(housing_model.parameters(), lr=learning_rate)

### Complete code for Train loop

In [ ]:
def housing_train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    loss = 0
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X=X.to(DEVICE)
        y=y.to(DEVICE)
        pred = model(X).flatten()
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)

            print(f"Train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

    return loss

### Complete code for Test loop

In [ ]:
def housing_test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X=X.to(DEVICE)
            y=y.to(DEVICE)
            pred = model(X).flatten()
            test_loss += loss_fn(pred, y).item()
    print("Test loss (avg):",  test_loss)

    test_loss /= num_batches

    return test_loss

### Optimization Process

In [ ]:
for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train_loss = housing_train_loop(train_housing_data_loader, housing_model, housing_loss_fn, housing_optimizer)
    test_loss = housing_test_loop(test_housing_data_loader, housing_model, housing_loss_fn)

    print("Housing_Loss/train", train_loss)
    print("Housing_Loss/test", test_loss)

print("Done!")